<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - HPF F1 </h1> 
<h2 align="center">Alejo Paullier</h2> 

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender
from lenskit.algorithms.hpf import HPF as hpfl
from hpfrec import HPF
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK
np.seterr(divide='ignore', invalid='ignore')

ratings = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 1M\\ratings.dat', sep='::',engine='python',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer(features):
    all_recs = []
    test_data = []
    features=int(features)
    
    hpf = hpfl(features,verbose=False) # define algorithm
     
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('HPF', hpf, train, test))
    
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla1 = topn.RecListAnalysis()
    rla2 = topn.RecListAnalysis()
    rla1.add_metric(topn.precision)
    rla2.add_metric(topn.recall)
    prec = rla1.compute(all_recs, test_data)
    rec = rla2.compute(all_recs, test_data)
    F1 = ((prec['precision'].values*rec['recall'].values*2)/(prec['precision'].values+rec['recall'].values))
    F1 = np.nan_to_num(F1)
    return [F1.mean(),prec['precision'].mean(),rec['recall'].mean()]

In [3]:
array=[]
def objective(params):
    features = params['features']
    metric = model_trainer(features)
    array.append([features,metric[0],metric[1],metric[2]])
    return {'loss': -metric[0], 'status': STATUS_OK }

space={'features': hp.uniform('features', 1, 100)}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

**********************************                                                                                                                                                                                                          
Hierarchical Poisson Factorization                                                                                                                                                                                                          
**********************************                                                                                                                                                                                                          
Number of users: 6040                                                                                                                                                                                                                       
Number of items: 3703                               

Iteration 30 | train llk: -4127463 | train rmse: 2.7373                                                                                                                                                                                     
Iteration 40 | train llk: -3953546 | train rmse: 2.6984                                                                                                                                                                                     
Iteration 50 | train llk: -3859618 | train rmse: 2.6763                                                                                                                                                                                     
Iteration 60 | train llk: -3806319 | train rmse: 2.6637                                                                                                                                                                                     
Iteration 70 | train llk: -3774438 | train rmse: 2.6

Minutes taken (optimization part): 0.7                                                                                                                                                                                                      
**********************************                                                                                                                                                                                                          
Hierarchical Poisson Factorization                                                                                                                                                                                                          
**********************************                                                                                                                                                                                                          
Number of users: 6040                               

Iteration 20 | train llk: -4431433 | train rmse: 2.8127                                                                                                                                                                                     
Iteration 30 | train llk: -4112710 | train rmse: 2.7384                                                                                                                                                                                     
Iteration 40 | train llk: -3944439 | train rmse: 2.6979                                                                                                                                                                                     
Iteration 50 | train llk: -3865411 | train rmse: 2.6795                                                                                                                                                                                     
Iteration 60 | train llk: -3822941 | train rmse: 2.6

Final RMSE: 2.5413                                                                                                                                                                                                                          
Minutes taken (optimization part): 1.6                                                                                                                                                                                                      
**********************************                                                                                                                                                                                                          
Hierarchical Poisson Factorization                                                                                                                                                                                                          
**********************************                  

...

Allocating Phi matrix...                                                                                                                                                                                                                    
Initializing optimization procedure...                                                                                                                                                                                                      
Iteration 10 | train llk: -5322223 | train rmse: 2.9844                                                                                                                                                                                     
Iteration 20 | train llk: -4639477 | train rmse: 2.8466                                                                                                                                                                                     
Iteration 30 | train llk: -4310901 | train rmse: 2.7

Iteration 100 | train llk: -4068141 | train rmse: 2.7361                                                                                                                                                                                    
                                                                                                                                                                                                                                            

Optimization finished
Final log-likelihood: -4068141                                                                                                                                                                                                              
Final RMSE: 2.7361                                                                                                                                                                                                                          
Minutes taken (optimization p

Initializing parameters...                                                                                                                                                                                                                  
Allocating Phi matrix...                                                                                                                                                                                                                    
Initializing optimization procedure...                                                                                                                                                                                                      
Iteration 10 | train llk: -5224544 | train rmse: 2.9742                                                                                                                                                                                     
Iteration 20 | train llk: -4475858 | train rmse: 2.8

Iteration 90 | train llk: -3861883 | train rmse: 2.6814                                                                                                                                                                                     
Iteration 100 | train llk: -3854790 | train rmse: 2.6800                                                                                                                                                                                    
                                                                                                                                                                                                                                            

Optimization finished
Final log-likelihood: -3854790                                                                                                                                                                                                              
Final RMSE: 2.6800           

KeyboardInterrupt: 

In [4]:
array

[[19.077547349779866,
  0.17844805155727547,
  0.1355976821192023,
  0.4793883594501818],
 [47.52563386162556,
  0.19159265916735352,
  0.1445662251655607,
  0.522021729547915],
 [47.10781626255315,
  0.19179312227508402,
  0.1446605960264872,
  0.5234484473675236],
 [53.637289870556785,
  0.1936047440448774,
  0.14594370860926872,
  0.5290638232928254],
 [22.185084079718987,
  0.18182378376403519,
  0.13792549668873877,
  0.49106935532354984],
 [88.7356418981237, 0.1958232250382671, 0.14735264900662, 0.536638100994929],
 [7.43292398647732,
  0.15450605404147452,
  0.11849006622516292,
  0.40941136972512415],
 [49.286502607444454,
  0.1917529911721446,
  0.1447599337748323,
  0.5227301426654849],
 [59.943138340508575,
  0.19376600270462563,
  0.14605960264900408,
  0.5286073595508053],
 [11.072116943094395,
  0.16558095378711518,
  0.1264801324503287,
  0.4387965249146988]]

In [6]:
array

[[47.30866940974619,
  0.19164721631603127,
  0.1446953642384076,
  0.5220162275907696],
 [41.52230064579835,
  0.18953624136404532,
  0.1433394039735071,
  0.5133453641274501],
 [59.69833719672212,
  0.19291957513561334,
  0.14554470198675254,
  0.5263441168754109],
 [32.129395761178046,
  0.18644429367941,
  0.1409884105960244,
  0.5062525664230456],
 [56.70003973045757,
  0.192803543354186,
  0.14547682119205085,
  0.5237277723224254],
 [91.0387491673498,
  0.19565044409022694,
  0.14736920529801062,
  0.5345737037436442],
 [20.816221417560428,
  0.1804875472464755,
  0.13705463576158658,
  0.4836228415591789],
 [95.20826194720584,
  0.1956227535584849,
  0.1472168874172156,
  0.5370291377825029]]

In [4]:
array

[[8.972390868568592,
  0.1572455480235318,
  0.12061589403973222,
  0.414634164125354],
 [81.54631579189218,
  0.19507463485869306,
  0.14690894039734867,
  0.5316859256694099]]

In [10]:
array # 695.13

[[44.385571242734684,
  0.19013624880657543,
  0.14358940397350725,
  0.516975289171592],
 [10.035241295851108,
  0.162884151364012,
  0.12459271523178568,
  0.43192822873744235],
 [34.70798253069235,
  0.1885847712740532,
  0.142504966887415,
  0.5122270584253068],
 [8.42460102322623,
  0.15702734267788718,
  0.12046523178807704,
  0.4130966535631921],
 [10.47932709540745,
  0.16188448839374384,
  0.1238493377483416,
  0.4296438449725755],
 [64.78148748342363,
  0.19338961072065364,
  0.14580629139072568,
  0.5277211731586415],
 [49.186361774072324,
  0.19175064647097037,
  0.14465728476820885,
  0.5220555898087615],
 [33.23203141880538,
  0.1884237576397993,
  0.14260596026489747,
  0.5101710192605995],
 [90.488309939969,
  0.19680366634483595,
  0.1477764900662231,
  0.5406376712536342],
 [12.414981571971072,
  0.1683363074212941,
  0.1282980132450308,
  0.4512036236250636],
 [21.97522004416283,
  0.18004144791206067,
  0.1366970198675475,
  0.4837689683699587],
 [66.90667006812063,

In [8]:
array #613.19

[[10.735926922657994,
  0.16298003414325704,
  0.12495364238410327,
  0.4296819417312323],
 [60.670780046242555,
  0.19334630185768684,
  0.14572350993377226,
  0.5273375508815963],
 [66.25638898111882,
  0.1944135869032577,
  0.14643543046357335,
  0.5323628580610165],
 [17.31201669104983,
  0.17606007162355367,
  0.13377317880794404,
  0.4731508360437865],
 [61.06161823941286,
  0.19433209181510616,
  0.14636920529801042,
  0.5315672138232064],
 [31.499035516922003,
  0.18626127341798412,
  0.14096854304635495,
  0.5047194426877768],
 [16.87261059778895,
  0.1743587837698678,
  0.132854304635759,
  0.46483422786716905],
 [56.45444235042311,
  0.1925565491469125,
  0.14520529801324214,
  0.5251035354325777],
 [54.71116235178047,
  0.19309253080720062,
  0.14566225165562624,
  0.5256585110877049],
 [71.28394561559398,
  0.19515653227532126,
  0.14702814569536168,
  0.53246473000333],
 [78.62789607781403,
  0.19562558214173742,
  0.1473592715231764,
  0.5340072977648516]]

In [6]:
array

[[77.29526828703706,
  0.19406927567745133,
  0.14625827814569264,
  0.5303332410636098],
 [66.20533312005169,
  0.19537140240345238,
  0.14706953642383844,
  0.5349374042445353],
 [59.72587413937597,
  0.19322441313528887,
  0.14580132450330857,
  0.5273272338475867],
 [8.154780756952345,
  0.15679853363355,
  0.12040397350993091,
  0.41188778997769016],
 [71.65680163235449,
  0.19388972787664657,
  0.14616225165562652,
  0.5283412191811596],
 [27.201874222947968,
  0.18435266658236502,
  0.13965894039734825,
  0.4975850131153666],
 [91.66446286184055,
  0.1963950697118542,
  0.14767218543046118,
  0.539212510896262],
 [60.97264502811718,
  0.19350784647474167,
  0.14585264900662037,
  0.5280935114216607],
 [18.06180069773852,
  0.17794009540216812,
  0.1352301324503283,
  0.4772480039763902]]

In [4]:
array

[[74.20223397804803,
  0.1948423101276518,
  0.14680132450330882,
  0.5309711644491925],
 [50.76344673213608,
  0.1918288270631377,
  0.14470364238410294,
  0.5235272942499897],
 [43.045619363175135,
  0.19093187755190497,
  0.14439403973509676,
  0.5171759419926713],
 [29.64350761208667,
  0.18533754534390046,
  0.1403360927152289,
  0.5014737319441868]]

In [6]:
array #685.06

[[23.45502753155876,
  0.1827203265543518,
  0.13851986754966683,
  0.49347188167786477],
 [77.07670941476934,
  0.19505180489303378,
  0.14681291390728188,
  0.5334491935899525],
 [30.868870313166344,
  0.18613467306660642,
  0.14094039735099106,
  0.5032906149540012],
 [56.518700923702156,
  0.19215182998238015,
  0.1448990066225139,
  0.5245691703280579],
 [81.36118520579292,
  0.1958214991151334,
  0.14722682119205058,
  0.5371640352672722],
 [82.00731411970068,
  0.19489088957383255,
  0.14673675496688493,
  0.5329967805063097],
 [51.673373104295365,
  0.19339270333499214,
  0.145768211920527,
  0.5281364923779537],
 [55.70567406888753,
  0.19342326083356945,
  0.1457764900662222,
  0.5280944766055822],
 [32.3751724303125,
  0.18630752905953035,
  0.14108112582781193,
  0.5048024723011051],
 [51.95410437807128,
  0.19227486879535666,
  0.14504966887416998,
  0.5253070880829375],
 [87.4132130890722,
  0.19469068891012484,
  0.14665562913907004,
  0.5328206269727066],
 [67.664675276

In [4]:
array #492.76

[[12.43633086294489,
  0.16815499195566208,
  0.1282582781456926,
  0.4465565956178724],
 [60.961464006994504,
  0.19371440470813084,
  0.14588907284767974,
  0.5299654502987055],
 [92.2108692342796,
  0.1950778258782543,
  0.1467483443708582,
  0.5362719533060206],
 [29.43601796628395,
  0.1860608390286294,
  0.14081456953642163,
  0.5019053724003469],
 [59.41431675107715,
  0.1936801649805945,
  0.14600331125827562,
  0.5286349084000593],
 [46.96913504077334,
  0.19091680600399052,
  0.1443079470198652,
  0.5186025017215138],
 [44.01377727890499,
  0.19025222068004818,
  0.14374834437085818,
  0.5158483621889246]]

In [7]:
metric = pd.DataFrame(array)
metric.columns = ['features','F1','Precision','Recall']
metric.head(5)

,features,F1,Precision,Recall
0,12.436331,0.168155,0.128258,0.446557
1,60.961464,0.193714,0.145889,0.529965
2,92.210869,0.195078,0.146748,0.536272
3,29.436018,0.186061,0.140815,0.501905
4,59.414317,0.193680,0.146003,0.528635


In [8]:
metric.sort_values(by=['F1'],ascending=False).head(10)

,features,F1,Precision,Recall
75,90.488310,0.196804,0.147776,0.540638
27,98.696944,0.196717,0.147934,0.538795
87,97.640622,0.196418,0.147719,0.537365
53,91.664463,0.196395,0.147672,0.539213
88,99.683750,0.196390,0.147821,0.537758
79,92.791772,0.196380,0.147788,0.537552
30,99.635798,0.196330,0.147581,0.539876
33,99.986690,0.196312,0.147730,0.537380
29,98.542837,0.196212,0.147783,0.536486
31,98.015341,0.196200,0.147515,0.538145


In [9]:
metric.to_csv('metric_F1.csv', index=False)

In [5]:
array = [[12.43633086294489,
  0.16815499195566208,
  0.1282582781456926,
  0.4465565956178724],
 [60.961464006994504,
  0.19371440470813084,
  0.14588907284767974,
  0.5299654502987055],
 [92.2108692342796,
  0.1950778258782543,
  0.1467483443708582,
  0.5362719533060206],
 [29.43601796628395,
  0.1860608390286294,
  0.14081456953642163,
  0.5019053724003469],
 [59.41431675107715,
  0.1936801649805945,
  0.14600331125827562,
  0.5286349084000593],
 [46.96913504077334,
  0.19091680600399052,
  0.1443079470198652,
  0.5186025017215138],
 [44.01377727890499,
  0.19025222068004818,
  0.14374834437085818,
  0.5158483621889246],
  [23.45502753155876,
  0.1827203265543518,
  0.13851986754966683,
  0.49347188167786477],
 [77.07670941476934,
  0.19505180489303378,
  0.14681291390728188,
  0.5334491935899525],
 [30.868870313166344,
  0.18613467306660642,
  0.14094039735099106,
  0.5032906149540012],
 [56.518700923702156,
  0.19215182998238015,
  0.1448990066225139,
  0.5245691703280579],
 [81.36118520579292,
  0.1958214991151334,
  0.14722682119205058,
  0.5371640352672722],
 [82.00731411970068,
  0.19489088957383255,
  0.14673675496688493,
  0.5329967805063097],
 [51.673373104295365,
  0.19339270333499214,
  0.145768211920527,
  0.5281364923779537],
 [55.70567406888753,
  0.19342326083356945,
  0.1457764900662222,
  0.5280944766055822],
 [32.3751724303125,
  0.18630752905953035,
  0.14108112582781193,
  0.5048024723011051],
 [51.95410437807128,
  0.19227486879535666,
  0.14504966887416998,
  0.5253070880829375],
 [87.4132130890722,
  0.19469068891012484,
  0.14665562913907004,
  0.5328206269727066],
 [67.66467527629617,
  0.19466947511888785,
  0.1465165562913879,
  0.5318576434974159],
 [22.20894801722077,
  0.18074439040917378,
  0.1372566225165538,
  0.4856722274801938],
 [35.575961371681906,
  0.18800510525076075,
  0.14217218543046112,
  0.5098908161812268],
 [64.0075790217871,
  0.19404811207167355,
  0.14630794701986496,
  0.5292917290229128],
 [89.04147375933638,
  0.19591395383371416,
  0.14736589403973285,
  0.5365144634674125],
 [6.44667784783252,
  0.15080091291927733,
  0.11600165562913681,
  0.39726243999581845],
 [56.36239864164105,
  0.1921299045454586,
  0.14492384105960027,
  0.5246508499992127],
 [15.011630843440269,
  0.1728925431841056,
  0.13175496688741467,
  0.4594832498308789],
 [11.3334069696342,
  0.16541112789905957,
  0.12630132450330822,
  0.440142821640807],
 [98.69694425254082,
  0.1967167941096481,
  0.1479337748344346,
  0.5387953942310272],
 [94.9899721449657,
  0.19476407460098727,
  0.1465413907284748,
  0.5335345987001844],
 [98.54283680321802,
  0.1962123287093991,
  0.14778311258277868,
  0.5364859042731448],
 [99.6357976206433,
  0.19633009632643786,
  0.14758112582781194,
  0.5398763375365462],
 [98.01534062201232,
  0.19619956830306956,
  0.1475149006622494,
  0.5381446056573164],
 [75.40818231728151,
  0.19456472819799156,
  0.14658774834436833,
  0.5314332958238073],
 [99.98668996766342,
  0.1963116421874969,
  0.14773013245032857,
  0.5373799499766162],
 [42.445833842021116,
  0.1894466419834623,
  0.14314569536423594,
  0.5143588587952973],
 [68.56179639943609,
  0.19579527323823204,
  0.1472947019867521,
  0.5369695945132427],
 [90.3379707327344,
  0.1961107084005296,
  0.14753476821191835,
  0.5370057470736985],
 [73.48513005731954,
  0.19464995854318515,
  0.14651324503310986,
  0.5329234171793316],
 [82.67381580638437,
  0.19607786289820006,
  0.1475695364238382,
  0.5367508193243188],
 [93.32163828866177,
  0.19576758413746592,
  0.1471672185430437,
  0.536528417051594],
 [83.3337969401316,
  0.19503176534516922,
  0.14675827814569326,
  0.5341878360841802],
 [40.24154255860055,
  0.1902593718253815,
  0.14362748344370574,
  0.5164585113098266],
 [65.37479945395933,
  0.19487157451270054,
  0.14677649006622243,
  0.5331419880380815],
    [74.20223397804803,
  0.1948423101276518,
  0.14680132450330882,
  0.5309711644491925],
 [50.76344673213608,
  0.1918288270631377,
  0.14470364238410294,
  0.5235272942499897],
 [43.045619363175135,
  0.19093187755190497,
  0.14439403973509676,
  0.5171759419926713],
 [29.64350761208667,
  0.18533754534390046,
  0.1403360927152289,
  0.5014737319441868],
  [77.29526828703706,
  0.19406927567745133,
  0.14625827814569264,
  0.5303332410636098],
 [66.20533312005169,
  0.19537140240345238,
  0.14706953642383844,
  0.5349374042445353],
 [59.72587413937597,
  0.19322441313528887,
  0.14580132450330857,
  0.5273272338475867],
 [8.154780756952345,
  0.15679853363355,
  0.12040397350993091,
  0.41188778997769016],
 [71.65680163235449,
  0.19388972787664657,
  0.14616225165562652,
  0.5283412191811596],
 [27.201874222947968,
  0.18435266658236502,
  0.13965894039734825,
  0.4975850131153666],
 [91.66446286184055,
  0.1963950697118542,
  0.14767218543046118,
  0.539212510896262],
 [60.97264502811718,
  0.19350784647474167,
  0.14585264900662037,
  0.5280935114216607],
 [18.06180069773852,
  0.17794009540216812,
  0.1352301324503283,
  0.4772480039763902],
  [10.735926922657994,
  0.16298003414325704,
  0.12495364238410327,
  0.4296819417312323],
 [60.670780046242555,
  0.19334630185768684,
  0.14572350993377226,
  0.5273375508815963],
 [66.25638898111882,
  0.1944135869032577,
  0.14643543046357335,
  0.5323628580610165],
 [17.31201669104983,
  0.17606007162355367,
  0.13377317880794404,
  0.4731508360437865],
 [61.06161823941286,
  0.19433209181510616,
  0.14636920529801042,
  0.5315672138232064],
 [31.499035516922003,
  0.18626127341798412,
  0.14096854304635495,
  0.5047194426877768],
 [16.87261059778895,
  0.1743587837698678,
  0.132854304635759,
  0.46483422786716905],
 [56.45444235042311,
  0.1925565491469125,
  0.14520529801324214,
  0.5251035354325777],
 [54.71116235178047,
  0.19309253080720062,
  0.14566225165562624,
  0.5256585110877049],
 [71.28394561559398,
  0.19515653227532126,
  0.14702814569536168,
  0.53246473000333],
 [78.62789607781403,
  0.19562558214173742,
  0.1473592715231764,
  0.5340072977648516],
         [44.385571242734684,
  0.19013624880657543,
  0.14358940397350725,
  0.516975289171592],
 [10.035241295851108,
  0.162884151364012,
  0.12459271523178568,
  0.43192822873744235],
 [34.70798253069235,
  0.1885847712740532,
  0.142504966887415,
  0.5122270584253068],
 [8.42460102322623,
  0.15702734267788718,
  0.12046523178807704,
  0.4130966535631921],
 [10.47932709540745,
  0.16188448839374384,
  0.1238493377483416,
  0.4296438449725755],
 [64.78148748342363,
  0.19338961072065364,
  0.14580629139072568,
  0.5277211731586415],
 [49.186361774072324,
  0.19175064647097037,
  0.14465728476820885,
  0.5220555898087615],
 [33.23203141880538,
  0.1884237576397993,
  0.14260596026489747,
  0.5101710192605995],
 [90.488309939969,
  0.19680366634483595,
  0.1477764900662231,
  0.5406376712536342],
 [12.414981571971072,
  0.1683363074212941,
  0.1282980132450308,
  0.4512036236250636],
 [21.97522004416283,
  0.18004144791206067,
  0.1366970198675475,
  0.4837689683699587],
 [66.90667006812063,
  0.19381042623840436,
  0.14599503311258,
  0.5310896863687554],
 [92.79177216417953,
  0.19637993499004663,
  0.14778807947019637,
  0.5375516685691788],
 [34.31488968901357,
  0.1883219500336511,
  0.1423410596026465,
  0.5126029653697696],
 [86.38476715540196,
  0.19540297560835676,
  0.14702483443708367,
  0.5350533376794894],
 [57.281982112480875,
  0.19437315179090023,
  0.1464403973509909,
  0.5307288835591922],
 [33.57218428465307,
  0.1884658932371045,
  0.14250993377483162,
  0.5110163381099204],
 [24.23523018749937,
  0.18196887901633182,
  0.13807450331125545,
  0.48816284043091973],
 [49.99683531584119,
  0.19131330187831433,
  0.1443526490066203,
  0.5203429758028971],
 [19.954757984819505,
  0.17839665464912408,
  0.13557284768211633,
  0.47828621718415093],
 [97.64062179114882,
  0.196418492179089,
  0.14771854304635504,
  0.5373654460382004],
 [99.6837503356465,
  0.19639047685513097,
  0.1478211920529769,
  0.5377584134667457],
   [8.972390868568592,
  0.1572455480235318,
  0.12061589403973222,
  0.414634164125354],
 [81.54631579189218,
  0.19507463485869306,
  0.14690894039734867,
  0.5316859256694099],
         [47.30866940974619,
  0.19164721631603127,
  0.1446953642384076,
  0.5220162275907696],
 [41.52230064579835,
  0.18953624136404532,
  0.1433394039735071,
  0.5133453641274501],
 [59.69833719672212,
  0.19291957513561334,
  0.14554470198675254,
  0.5263441168754109],
 [32.129395761178046,
  0.18644429367941,
  0.1409884105960244,
  0.5062525664230456],
 [56.70003973045757,
  0.192803543354186,
  0.14547682119205085,
  0.5237277723224254],
 [91.0387491673498,
  0.19565044409022694,
  0.14736920529801062,
  0.5345737037436442],
 [20.816221417560428,
  0.1804875472464755,
  0.13705463576158658,
  0.4836228415591789],
 [95.20826194720584,
  0.1956227535584849,
  0.1472168874172156,
  0.5370291377825029],
         [19.077547349779866,
  0.17844805155727547,
  0.1355976821192023,
  0.4793883594501818],
 [47.52563386162556,
  0.19159265916735352,
  0.1445662251655607,
  0.522021729547915],
 [47.10781626255315,
  0.19179312227508402,
  0.1446605960264872,
  0.5234484473675236],
 [53.637289870556785,
  0.1936047440448774,
  0.14594370860926872,
  0.5290638232928254],
 [22.185084079718987,
  0.18182378376403519,
  0.13792549668873877,
  0.49106935532354984],
 [88.7356418981237, 0.1958232250382671, 0.14735264900662, 0.536638100994929]
]